In [1]:
import torch

from childnet import ChildNetwork
from cifar10_training import Cifar10Trainer
import controller_training
from controller import ControllerNetwork
from utils import layer_sampler, connection_sampler, exponential_moving_average, moving_average

In [2]:
outer_iterations = 310 #310
controller_iterations = 1  #2000
child_iterations = 4 #2
##
hidden_size = 100
num_layer_types = 4
num_layers = 8
num_filters = 32
num_classes = 10  # cifar-10 dataset
grad_clip_value = 0.15
##
val_accuracy_list = []


train_time_list = []

In [3]:

cifar10_trainer = Cifar10Trainer(batch_size=128)

what is the device? cpu or gpu ?  cuda
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
controller = ControllerNetwork(hidden_size, num_layers, num_layer_types)
load_previous_network_weights = None
#
for i in range(outer_iterations):
    print("ITERATION NUMBER %d" % (i+1))
    ############## sample model ##############
    print('Controller is sampling architecture...')
    soft_1, soft_2 = controller()
    #
    print(soft_1)
    layers_types = layer_sampler(soft_1)
    print(layers_types)
    # print(layers_types.shape)
    # take log after sampling (we couldn't do it before sampling)
    soft_1 = torch.log(soft_1)
    #
    skip_connections = connection_sampler(soft_2)
    ############## initialize weights ##############
    print('Creating new child network...')
    if i == 0:
        load_previous_network_weights = False
    child_net = ChildNetwork(layers_types, skip_connections, load_previous_network_weights,
                             num_hidden_layers=num_layers,
                             num_filters=num_filters, num_classes=num_classes)
    load_previous_network_weights = True

    ############## train model ##############
    print('Training child network...')
    # if(i > 0):
    #     child_net.load_state_dict(torch.load('weights.pth'))
    val_accuracy, new_train_time_list = cifar10_trainer.train(child_net, no_epoch=child_iterations)
    val_accuracy_list.append(val_accuracy)
    train_time_list.extend(new_train_time_list)
    ############## update weights ##############
    print('Saving child network\'s weights...')
    # torch.save(child_net.state_dict(), 'weights.pth')

    ############## train controller ##############
    print('Training controller network')
    baseline = moving_average(val_accuracy_list)
    multiplier = val_accuracy - baseline # (reward - baseline)
    print('val_accuracy is = ', val_accuracy)
    print('baseline is = ', baseline)
    print('multiplier is = ', multiplier)
    controller_training.train(controller, soft_1, soft_2, layers_types, skip_connections, multiplier, grad_clip_value=grad_clip_value, no_epoch=controller_iterations)
    ##############################################
    print('##############################################')
    print('##############################################')

ITERATION NUMBER 1
Controller is sampling architecture...
tensor([[[0.2400, 0.2453, 0.2438, 0.2709]],

        [[0.2407, 0.2444, 0.2429, 0.2720]],

        [[0.2413, 0.2441, 0.2421, 0.2725]],

        [[0.2453, 0.2413, 0.2398, 0.2737]],

        [[0.2481, 0.2394, 0.2388, 0.2737]],

        [[0.2519, 0.2369, 0.2379, 0.2734]],

        [[0.2545, 0.2353, 0.2375, 0.2727]],

        [[0.2574, 0.2335, 0.2372, 0.2720]],

        [[0.2593, 0.2324, 0.2370, 0.2713]],

        [[0.2614, 0.2311, 0.2370, 0.2706]],

        [[0.2628, 0.2303, 0.2369, 0.2699]],

        [[0.2643, 0.2294, 0.2370, 0.2693]],

        [[0.2654, 0.2288, 0.2370, 0.2688]],

        [[0.2665, 0.2281, 0.2371, 0.2683]],

        [[0.2674, 0.2277, 0.2372, 0.2678]],

        [[0.2683, 0.2271, 0.2373, 0.2673]]], grad_fn=<CatBackward>)
tensor([[[0., 0., 0., 1.]],

        [[0., 0., 0., 0.]],

        [[0., 0., 1., 0.]],

        [[0., 0., 0., 0.]],

        [[0., 1., 0., 0.]],

        [[0., 0., 0., 0.]],

        [[0., 0., 0., 1.]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(val_accuracy_list)
plt.figure()
plt.plot(train_time_list)